<a href="https://colab.research.google.com/github/BSniegowski/ML-uni_course/blob/main/project/mdd_eeg_diagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!wget https://figshare.com/ndownloader/articles/4244171/versions/2

--2023-01-11 20:17:05--  https://figshare.com/ndownloader/articles/4244171/versions/2
Resolving figshare.com (figshare.com)... 54.229.244.39, 52.30.99.27, 2a05:d018:1f4:d003:1350:74d8:3c49:3fbd, ...
Connecting to figshare.com (figshare.com)|54.229.244.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 851639264 (812M) [application/zip]
Saving to: ‘2’

2                   100%[===================>] 812.19M  23.1MB/s    in 40s     

2023-01-11 20:17:45 (20.4 MB/s) - ‘2’ saved [851639264/851639264]



In [ ]:
!unzip 2

In [ ]:
!pip install mne

In [79]:
import os
import mne

mne.set_log_level('ERROR')


raws = []
for file in os.scandir('.'):
  _, ext = os.path.splitext(file.path)
  if ext == ".edf":
    raws.append(mne.io.read_raw_edf(file.path))

In [116]:
import torch
from torch.utils.data import Dataset

# at first let's ignore timestamps, that is treat the measurements as sequences
# instead of functions of time
class CustomDataset(Dataset):
  def __init__(self, raws, labels, transform=None):
    assert len(raws) == len(labels)
    self.data = [raw.load_data()[:,:][0] for raw in raws]
    self.labels = labels
    self.transform = transform

  def __len__(self):
    return len(self.labels)

  def __getitem__(self,idx):
    X, y = self.data[idx], self.labels[idx]
    if self.transform is not None:
      X = self.transform(X)
    X = torch.as_tensor(X[None, ...])
    return X, y

# def scale(X):
#   """Standard scaling of data along the last dimention.
    
#   Parameters
#   ----------
#   X : array, shape (n_channels, n_times)
#       The input signals.
        
#   Returns
#   -------
#   X_t : array, shape (n_channels, n_times)
#       The scaled signals.
#   """
#   X -= torch.mean(X, axis=1, keepdims=True)
#   return X / torch.std(X, axis=1, keepdims=True)

In [112]:
labels = [1 if "MDD" in raw.filenames[0] else 0 for raw in raws]

In [117]:
dataset = CustomDataset(raws, labels)

In [118]:
from torch.utils.data import random_split
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])